In [6]:
import os
import re

In [7]:
functionsREGEX = ["(scanf[ ]*\([ ]*\"%s\")|(scanf\(.{1,}[,][ ]*\".*%s.*\")",
                  "(gets[ ]*\()", 
                  "(strcpy[ ]*\(.{1,}[,].{1,}\))", 
                  "(memcpy[ ]*\(.{1,}[,].{1,}\))", 
                  "(memmove[ ]*\(.{1,}[,].{1,}\))", 
                  "(memset[ ]*\(.{1,}[,].{1,}\))", 
                  "(strncat[ ]*\(.{1,}[,].{1,}[,].{1,}\))", 
                  "(strcat[ ]*\(.{1,}[,].{1,}\))", 
                  "(strncpy[ ]*\(.{1,}[,].{1,}[,].{1,}\))", 
                  "(sprintf|vsprintf|swprintf|vswprintf|_stprintf|_vstprintf)([ ]*\(.{1,}[,])",
                  "(MultiByteToWideChar[ ]*\(.{1,}[,].{1,}[,].{1,}[,].{1,}[,].{1,}[,].{1,}\))",
                  "(strtrns[ ]*\(.{1,}[,].{1,}[,].{1,}[,].{1,}\))", 
                  "(realpath[ ]*\(.{1,}[,].{1,}\))", 
                  "(getopt|getopt_long)([ ]*\(.{1,}[,].{1,}[,].{1,}\))", 
                  "(getwd[ ]*\(.{1,}[,].{1,}\))", 
                  "(getchar|fgetc|getc|read|_gettc)(\(.*\))"]

In [8]:
folderPath = './Testdata/'
fileList = os.listdir(folderPath)

In [9]:
def findFunctionCall (fileList, functionsKeywords):
    fileDict = dict()
    for fileName in fileList:
        print('Checking for \"CWE-120: Buffer Copy without Checking Size of Input\" in file ', fileName)
        print('-----------------------------------------------------------------------------\n')
        fileDict[fileName] = {"secure": True}
        fileR = open(folderPath + fileName, 'r')
        lines = fileR.readlines()
        vunerableCount = 0
        lineNum = 0

        for line in lines:
            lineNum += 1
            for functionREGEX in functionsREGEX:
                functionREGEX = '(?<!\/\/)' + functionREGEX #Expression to ensure that line is not a comment 
                lineNoWhiteSpace = "".join(line.split()) #Remove spaces since negative lookbehind pattern needs a definitive width
                match = re.search(functionREGEX, lineNoWhiteSpace)
                if match:
                    coord = match.span()
                    if coord[0] > line.find('//'): #Explicitly check that the match is before the comment
                        vunerableCount += 1
                        fileDict[fileName].update({"line" + str(lineNum): line, "secure": False})
                        print('Possible vulnerability : \"CWE-120: Buffer Copy without Checking Size of Input\"')
                        print('Need inspection at Line #',lineNum,' :',line)
                        print()

        if vunerableCount == 0:
            print('No \"CWE-120: Buffer Copy without Checking Size of Input\" found in file: ',fileName)
        else:
            print(vunerableCount,'possible \"CWE-120: Buffer Copy without Checking Size of Input\" found in file: ',fileName)
        print('-----------------------------------------------------------------------------\n')
    return fileDict

In [10]:
findFunctionCall(fileList, functionsREGEX)

Checking for "CWE-120: Buffer Copy without Checking Size of Input" in file  example1.c
-----------------------------------------------------------------------------

Possible vulnerability : "CWE-120: Buffer Copy without Checking Size of Input"
Need inspection at Line # 12  :     scanf ("%s", last_name);


Possible vulnerability : "CWE-120: Buffer Copy without Checking Size of Input"
Need inspection at Line # 16  :     char c = getchar();


Possible vulnerability : "CWE-120: Buffer Copy without Checking Size of Input"
Need inspection at Line # 19  :         c = getchar();


Possible vulnerability : "CWE-120: Buffer Copy without Checking Size of Input"
Need inspection at Line # 24  :     gets(buf);


Possible vulnerability : "CWE-120: Buffer Copy without Checking Size of Input"
Need inspection at Line # 32  :     strcpy(buf, string);


5 possible "CWE-120: Buffer Copy without Checking Size of Input" found in file:  example1.c
-------------------------------------------------------------

{'example1.c': {'secure': False,
  'line12': '    scanf ("%s", last_name);\n',
  'line16': '    char c = getchar();\n',
  'line19': '        c = getchar();\n',
  'line24': '    gets(buf);\n',
  'line32': '    strcpy(buf, string);\n'},
 'example2.c': {'secure': False,
  'line13': '    scanf ( "%s", buf);\n',
  'line15': '    char c = getchar();\n',
  'line18': '        c = getchar();\n',
  'line24': '    gets ( buf );\n',
  'line32': '    strcpy ( buf , string);\n'},
 'example3.c': {'secure': False,
  'line13': '    if (memcpy(str2,str1, strlen(str1)))\n',
  'line28': '    if (memmove(str3+2,str3, strlen(str3)))\n',
  'line45': '    memset(a, 3, 5*sizeof(char));\n'}}